# Resnet33 for Real Time Automatic Modulation Classification

### Import Statements

In [1]:
import torch
import h5py
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import os
import numpy as np
import torch.utils.data as data
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
from pathlib import Path

### Define Module

In [2]:
class resnet33_new(torch.nn.Module):
    def __init__(self, channels):
        super(resnet33_new, self).__init__() #Consider only 1 residual stack layer
        self.conv1 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=2, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        self.maxpool = torch.nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv2 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        self.conv3 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        self.conv4 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        self.conv5 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        
        self.conv6 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        self.conv7 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        self.conv8 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        self.conv9 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        
        self.conv10 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        self.conv11 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        self.conv12 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        self.conv13 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        
        self.conv14 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        self.conv15 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        self.conv16 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        self.conv17 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        
        self.conv18 = torch.nn.Sequential(torch.nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=3, padding=1),torch.nn.BatchNorm1d(channels))
        
        self.classifier = torch.nn.Sequential(torch.nn.Linear(1024,128),
                                        #torch.nn.SELU(True),
                                        torch.nn.ReLU(True),
                                        #torch.nn.Dropout(),
                                        torch.nn.Linear(128,128),
                                        #torch.nn.SELU(True),
                                        torch.nn.ReLU(True),
                                        #torch.nn.Dropout(), 
                                        torch.nn.Linear(128,24))
                                        #torch.nn.Softmax(dim=1)) 
            
    def forward(self,x):
        #Residual Stack 1
        y1 = F.relu(self.conv1(x)) #1'st layer con
        y2 = F.relu(self.conv2(y1))
        y3 = self.maxpool(F.relu(self.conv3(y2) + y1))
        
        #Residual Stack 2
        y4 = self.conv4(y3)
        y5 = F.relu(self.conv5(y4))
        y6 = self.maxpool(F.relu(self.conv6(y5) + y4))
        
        #Residual Stack 3
        y4 = self.conv7(y6)
        y5 = F.relu(self.conv8(y4))
        y6 = self.maxpool(F.relu(self.conv9(y5) + y4))
        
        #Residual Stack 4
        y4 = self.conv10(y6)
        y5 = F.relu(self.conv11(y4))
        y6 = self.maxpool(F.relu(self.conv12(y5) + y4))
        
        #Residual Stack 5
        y4 = self.conv13(y6)
        y5 = F.relu(self.conv14(y4))
        y6 = self.maxpool(F.relu(self.conv15(y5) + y4))
        
        #Residual Stack 6
        y4 = self.conv16(y6)
        y5 = F.relu(self.conv17(y4))
        y6 = self.maxpool(F.relu(self.conv18(y5) + y4))
        
        y7 = torch.flatten(y6,1)
        y8 = self.classifier(y7)
        
        return y8
        
            

In [3]:
class H5Dataset(data.Dataset):

    def __init__(self, file_path):
        super(H5Dataset, self).__init__()
        h5_file = h5py.File(file_path,'r')
        self.X = h5_file['X']
        self.Y = h5_file['Y']
        self.Z = h5_file['Z']

    def __getitem__(self, index):
        #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        #print(index)
        Y = torch.from_numpy(self.Y[index])
        return torch.from_numpy(self.X[index]).float(),torch.argmax(Y),torch.from_numpy(self.Z[index]).float()
        #return self.data

    def __len__(self):
        return self.X.shape[0]

### Train Network

In [4]:
def train_model(net, train_loader, epochs, criterion, optimizer, device):
    running_loss = 0.0
    steps = 0
    print_every = 100
    net.train()
    train_losses = []
    epoch_start = 0
    correct = 0
    
    for epoch in range(epoch_start,epochs):
        for i, data in tqdm(enumerate(train_loader)):
            steps += 1
            signals, labels, snr = data
            signals, labels = signals.to(device), labels.to(device)
            #print(labels)
            signals_mean = torch.mean(signals, 1, keepdims=True)
            #print(signals.size())
            #print(signals_mean.size())
            signals = signals - signals_mean
            signals = torch.transpose(signals, 1, 2)
            
            optimizer.zero_grad()
            outputs = net.forward(signals)
            correct += torch.sum((torch.argmax(outputs,dim=1) == labels)).item()
            #print(torch.argmax(outputs,dim=1)==labels, correct)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

            if steps % print_every == 0:
                train_losses.append(running_loss/len(train_loader))
                print(f"Epoch {epoch+1}/{epochs}.. "
                                  f"Train loss: {running_loss/print_every:.3f}.. ")
                print('Accuracy {:f}'.format(correct/(48*print_every)))
                running_loss = 0
                correct = 0
                #net.train()
                print("Steps completed: ",steps)
                
        PATH = './resnet33_radioml_epoch_{:d}.pth'.format(epoch)
        torch.save(net.state_dict(), PATH) 
    return

In [5]:
#loader_params = {'batch_size': 64, 'shuffle': True, 'num_workers': 6}

input_data = H5Dataset("/dataset/201801/GOLD_XYZ_OSC.0001_1024.hdf5")
dataset_size = len(input_data)
print(dataset_size)

"""
indices = list(range(dataset_size))
split = int(np.floor(0.2 * dataset_size))
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
print(len(train_indices))
print(len(val_indices))


f = open('val_idx.txt', 'r')

val_indices = []
for line in f.readlines():
    val_indices.append(int(line))
    
print(len(val_indices))
f.close()

f = open('train_idx.txt','r')

train_indices = []
for line in f.readlines():
    train_indices.append(int(line))
    
print(len(train_indices))
f.close()
"""
indices = []
for i in range (0, 24):
    temp = list(range(106496*i +4096*13, 106496*i + 4096*26)) #Get only SNR > 6dB
    indices.extend(temp)

print(len(temp))
print(len(indices))

split = int(np.floor(0.2*len(indices)))
np.random.shuffle(indices)
#print(indices[0:50000])
train_indices, val_indices = indices[split:], indices[:split]

print(train_indices[:100])
print(val_indices[:100])

train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
validation_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(input_data, batch_size=48,shuffle=False, sampler=train_sampler,num_workers=48,pin_memory=True)
validation_loader = torch.utils.data.DataLoader(input_data, batch_size=48,shuffle=False, sampler=validation_sampler, num_workers=48, pin_memory=True)

# setting up for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

#PATH = './resnet33_radioml_epoch_20.pth'
#torch.save(model.state_dict(), PATH) 

#model2 = resnet33_new(channels=64)
#model2.to(device)
#model2.load_state_dict(torch.load(PATH))

#epochs = 15

#train_model(model2, train_loader, epochs, criterion, optimizer,device)

net = resnet33_new(channels=64)
net = net.to(device)
#net.load_state_dict(torch.load(PATH))
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9,0.999))
#checkpoint_file = '/datamount/vgg_pytorch.pth'
epochs = 30

# train part
train_model(net, train_loader, epochs, criterion, optimizer,device)

2555904
53248
1277952
[505001, 638444, 2193792, 719133, 821901, 1132981, 1443590, 522311, 1580904, 1556031, 2551912, 311639, 1586301, 1477505, 1770100, 621663, 1130402, 2299317, 313611, 2437287, 932119, 1150478, 1057659, 309823, 1783206, 738028, 1233470, 2424012, 1456462, 1864173, 210909, 2194119, 1798642, 2196659, 721707, 2401708, 636437, 86860, 1761140, 1163736, 2226004, 1557734, 2441873, 2291169, 1905000, 1475545, 2311486, 1156841, 1772446, 1061859, 2521491, 292359, 2121051, 59471, 1014178, 500135, 2104314, 176801, 2438665, 1059780, 1164582, 807163, 2325514, 1880519, 283286, 1884370, 2519676, 2201275, 1572682, 1664136, 1759830, 186024, 518174, 1482981, 1444061, 311415, 2081730, 1374004, 1225459, 1446509, 2187011, 1277035, 1571949, 1885876, 1571960, 491942, 2184563, 1549089, 398968, 1578680, 1038459, 1162286, 2014235, 1697347, 104847, 813075, 1274990, 185778, 1808126, 2293236]
[1596858, 521150, 1053835, 2101719, 1554136, 846210, 2305533, 1982107, 618783, 1017518, 425338, 299909, 2518

101it [00:16, 15.03it/s]

Epoch 1/30.. Train loss: 2.153.. 
Accuracy 0.255000
Steps completed:  100


202it [00:27, 16.44it/s]

Epoch 1/30.. Train loss: 1.390.. 
Accuracy 0.407500
Steps completed:  200


246it [00:31, 15.14it/s]Exception in thread Thread-4:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/pin_memory.py", line 25, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/opt/conda/lib/python3.7/site-packages/torch/multiprocessing/reductions.py", line 294, in rebuild_storage_fd
    fd = df.detach()
  File "/opt/conda/lib/python3.7/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/opt/conda/lib/python3.7/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=proc

KeyboardInterrupt: 

### Test Network

In [19]:
PATH = './resnet33_radioml_epoch_29.pth'
#torch.save(model.state_dict(), PATH) 

model1 = resnet33_new(channels=64)
model1.to(device)
model1.load_state_dict(torch.load(PATH))

correct = 0
total = 0
"""
f = open('val_idx.txt', 'r')

new_val_indices = []
for line in f.readlines():
    new_val_indices.append(int(line))
    
print(len(new_val_indices))
f.close()
"""
snr_accuracy = dict()
for i in range(6,32,2):
    k = str(i)
    snr_accuracy[k] = [0,0]
    
#new_validations_sampler = torch.utils.data.SubsetRandomSampler(new_val_indices)
    
new_validation_loader = torch.utils.data.DataLoader(input_data, batch_size=1,shuffle=False, sampler=validation_sampler, num_workers=48, pin_memory=True)

print(len(validation_loader))

model1.eval()
for i,data in tqdm(enumerate(new_validation_loader)):
    signals, labels, snr = data
    signals, labels = signals.to(device), labels.to(device)
    #print(labels)
    signals_mean = torch.mean(signals, 1, keepdims=True)
    #print(signals.size())
    #print(signals_mean.size())
    signals = signals - signals_mean
    signals = torch.transpose(signals, 1, 2)

    #optimizer.zero_grad()
    outputs = model1.forward(signals)
    correct += torch.sum((torch.argmax(outputs,dim=1) == labels)).item()
    snr_value = snr.item()
    #print(snr_value)
    snr_accuracy[str(int(snr_value))][0] += torch.sum((torch.argmax(outputs,dim=1) == labels)).item()
    snr_accuracy[str(int(snr_value))][1] += 1
    
    #print(snr_accuracy)
    #print(torch.argmax(outputs,dim=1)==labels, correct)
    #loss = criterion(outputs,labels)
    #loss.backward()
    #optimizer.step()
    #predicted = torch.argmax(outputs, 1)#Calculate predicted value
    #print(predicted, labels)
    total += 1 #Increase by 1 batch
    #print(correct, total)
    #print(predicted == labels)
    #correct += torch.sum((predicted == labels)) #Increase if correct

print('Accuracy of the network on the test images: %f %%' % (
    100 * correct / total))
    

5325





0it [00:00, ?it/s]


1it [00:00,  3.49it/s]


2it [00:00,  3.87it/s]


9it [00:00,  5.38it/s]


15it [00:00,  7.40it/s]


22it [00:00, 10.12it/s]


31it [00:00, 13.76it/s]


39it [00:00, 18.31it/s]


47it [00:01, 23.74it/s]


55it [00:01, 29.87it/s]


63it [00:01, 36.75it/s]


72it [00:01, 44.23it/s]


80it [00:01, 47.14it/s]


88it [00:01, 53.31it/s]


96it [00:01, 58.61it/s]


105it [00:01, 64.58it/s]


113it [00:01, 67.45it/s]


121it [00:02, 70.12it/s]


129it [00:02, 72.63it/s]


137it [00:02, 74.28it/s]


145it [00:02, 75.09it/s]


153it [00:02, 75.69it/s]


161it [00:02, 76.35it/s]


170it [00:02, 78.90it/s]


178it [00:02, 78.37it/s]


186it [00:02, 78.20it/s]


194it [00:03, 77.69it/s]


202it [00:03, 77.50it/s]


210it [00:03, 69.27it/s]


218it [00:03, 71.41it/s]


226it [00:03, 73.07it/s]


234it [00:03, 74.62it/s]


242it [00:03, 75.39it/s]


250it [00:03, 75.74it/s]


258it [00:03, 76.66it/s]


266it [00:03, 76.88it/s]


274it [00:04, 77.08it/s]


282it [00:04, 77.15it

3584it [00:49, 75.81it/s]


3592it [00:49, 75.63it/s]


3600it [00:50, 74.55it/s]


3608it [00:50, 75.11it/s]


3616it [00:50, 74.76it/s]


3624it [00:50, 66.46it/s]


3632it [00:50, 68.62it/s]


3640it [00:50, 70.80it/s]


3648it [00:50, 72.81it/s]


3656it [00:50, 72.99it/s]


3664it [00:50, 73.00it/s]


3672it [00:51, 73.85it/s]


3681it [00:51, 77.66it/s]


3689it [00:51, 76.90it/s]


3697it [00:51, 76.64it/s]


3705it [00:51, 66.47it/s]


3712it [00:51, 66.47it/s]


3719it [00:51, 65.58it/s]


3726it [00:51, 66.45it/s]


3734it [00:51, 69.33it/s]


3744it [00:52, 76.27it/s]


3752it [00:52, 76.28it/s]


3760it [00:52, 76.11it/s]


3768it [00:52, 76.39it/s]


3776it [00:52, 76.22it/s]


3784it [00:52, 68.36it/s]


3792it [00:52, 70.39it/s]


3800it [00:52, 71.69it/s]


3808it [00:52, 72.55it/s]


3816it [00:53, 73.46it/s]


3824it [00:53, 74.06it/s]


3832it [00:53, 73.56it/s]


3840it [00:53, 73.77it/s]


3848it [00:53, 74.29it/s]


3856it [00:53, 67.01it/s]


3864it [00:53, 68.74

7808it [01:39, 74.82it/s]


7816it [01:39, 66.26it/s]


7824it [01:39, 68.54it/s]


7832it [01:39, 70.36it/s]


7840it [01:39, 72.31it/s]


7848it [01:39, 73.26it/s]


7856it [01:39, 74.25it/s]


7864it [01:39, 74.94it/s]


7872it [01:40, 75.64it/s]


7880it [01:40, 75.34it/s]


7888it [01:40, 66.77it/s]


7896it [01:40, 69.52it/s]


7904it [01:40, 71.36it/s]


7912it [01:40, 72.07it/s]


7920it [01:40, 73.07it/s]


7928it [01:40, 74.08it/s]


7936it [01:40, 74.31it/s]


7944it [01:41, 73.31it/s]


7952it [01:41, 73.95it/s]


7960it [01:41, 66.55it/s]


7968it [01:41, 69.04it/s]


7976it [01:41, 71.33it/s]


7984it [01:41, 72.30it/s]


7992it [01:41, 72.20it/s]


8000it [01:41, 73.35it/s]


8009it [01:41, 76.62it/s]


8017it [01:42, 76.05it/s]


8025it [01:42, 75.81it/s]


8033it [01:42, 67.29it/s]


8040it [01:42, 67.61it/s]


8048it [01:42, 70.73it/s]


8056it [01:42, 71.97it/s]


8064it [01:42, 72.99it/s]


8072it [01:42, 73.66it/s]


8080it [01:42, 73.82it/s]


8088it [01:43, 73.94

11442it [02:27, 73.66it/s]


11450it [02:27, 67.13it/s]


11458it [02:27, 69.40it/s]


11466it [02:27, 70.91it/s]


11474it [02:28, 71.91it/s]


11482it [02:28, 73.28it/s]


11490it [02:28, 74.47it/s]


11498it [02:28, 74.40it/s]


11506it [02:28, 73.97it/s]


11514it [02:28, 74.09it/s]


11522it [02:28, 66.84it/s]


11530it [02:28, 69.73it/s]


11538it [02:28, 70.93it/s]


11546it [02:29, 72.33it/s]


11554it [02:29, 72.77it/s]


11562it [02:29, 73.71it/s]


11570it [02:29, 74.04it/s]


11578it [02:29, 74.66it/s]


11586it [02:29, 73.59it/s]


11594it [02:29, 66.46it/s]


11601it [02:29, 66.84it/s]


11609it [02:29, 68.98it/s]


11616it [02:30, 67.89it/s]


11624it [02:30, 70.32it/s]


11632it [02:30, 72.51it/s]


11640it [02:30, 73.73it/s]


11648it [02:30, 73.25it/s]


11656it [02:30, 73.57it/s]


11664it [02:30, 74.28it/s]


11672it [02:30, 67.33it/s]


11679it [02:30, 67.95it/s]


11687it [02:31, 69.64it/s]


11695it [02:31, 71.38it/s]


11703it [02:31, 72.50it/s]


11711it [02:31

...

243653it [51:20, 71.97it/s]


243662it [51:20, 76.02it/s]


243670it [51:20, 75.12it/s]


243678it [51:20, 74.42it/s]


243690it [51:20, 83.62it/s]


243699it [51:20, 73.59it/s]


243707it [51:20, 72.78it/s]


243715it [51:21, 73.07it/s]


243723it [51:21, 72.99it/s]


243731it [51:21, 74.47it/s]


243739it [51:21, 74.66it/s]


243747it [51:21, 66.62it/s]


243755it [51:21, 68.96it/s]


243763it [51:21, 70.51it/s]


243771it [51:21, 71.96it/s]


243779it [51:21, 73.12it/s]


243787it [51:22, 73.13it/s]


243795it [51:22, 73.03it/s]


243803it [51:22, 73.27it/s]


243811it [51:22, 65.92it/s]


243819it [51:22, 68.60it/s]


243826it [51:22, 68.91it/s]


243833it [51:22, 69.21it/s]


243841it [51:22, 71.52it/s]


243849it [51:22, 72.75it/s]


243857it [51:23, 73.10it/s]


243865it [51:23, 73.16it/s]


243873it [51:23, 73.28it/s]


243881it [51:23, 73.67it/s]


243890it [51:23, 69.20it/s]


243897it [51:23, 69.38it/s]


243905it [51:23, 70.67it/s]


243914it [51:23, 75.16it/s]


243922it [

247772it [52:05, 77.34it/s]


247780it [52:06, 76.60it/s]


247788it [52:06, 68.00it/s]


247796it [52:06, 69.57it/s]


247804it [52:06, 70.70it/s]


247812it [52:06, 72.33it/s]


247820it [52:06, 73.69it/s]


247828it [52:06, 74.33it/s]


247836it [52:06, 73.19it/s]


247844it [52:06, 72.78it/s]


247852it [52:07, 66.18it/s]


247860it [52:07, 68.99it/s]


247868it [52:07, 71.60it/s]


247877it [52:07, 75.35it/s]


247885it [52:07, 74.46it/s]


247893it [52:07, 74.40it/s]


247901it [52:07, 74.78it/s]


247909it [52:07, 75.89it/s]


247917it [52:07, 75.81it/s]


247925it [52:08, 75.36it/s]


247933it [52:08, 66.68it/s]


247942it [52:08, 71.41it/s]


247950it [52:08, 72.29it/s]


247958it [52:08, 72.97it/s]


247966it [52:08, 74.64it/s]


247974it [52:08, 74.83it/s]


247982it [52:08, 73.90it/s]


247990it [52:08, 74.70it/s]


247998it [52:09, 75.38it/s]


248006it [52:09, 75.47it/s]


248014it [52:09, 67.91it/s]


248022it [52:09, 69.81it/s]


248030it [52:09, 71.62it/s]


248038it [

251183it [52:52, 69.78it/s]


251191it [52:52, 70.41it/s]


251199it [52:53, 71.59it/s]


251207it [52:53, 73.13it/s]


251215it [52:53, 74.36it/s]


251223it [52:53, 74.65it/s]


251231it [52:53, 67.33it/s]


251238it [52:53, 67.50it/s]


251246it [52:53, 70.02it/s]


251254it [52:53, 71.44it/s]


251262it [52:53, 71.47it/s]


251270it [52:53, 71.38it/s]


251278it [52:54, 72.55it/s]


251286it [52:54, 73.18it/s]


251294it [52:54, 66.02it/s]


251302it [52:54, 68.27it/s]


251309it [52:54, 68.66it/s]


251317it [52:54, 70.99it/s]


251325it [52:54, 72.40it/s]


251333it [52:54, 73.20it/s]


251341it [52:54, 74.19it/s]


251349it [52:55, 74.93it/s]


251357it [52:55, 74.64it/s]


251365it [52:55, 75.30it/s]


251373it [52:55, 65.21it/s]


251380it [52:55, 65.76it/s]


251388it [52:55, 68.27it/s]


251395it [52:55, 68.33it/s]


251402it [52:55, 68.12it/s]


251410it [52:55, 70.29it/s]


251418it [52:56, 71.90it/s]


251426it [52:56, 71.10it/s]


251434it [52:56, 65.13it/s]


251441it [

254648it [53:40, 62.71it/s]


254663it [53:40, 75.32it/s]


254675it [53:40, 55.36it/s]


254685it [53:41, 47.00it/s]


254699it [53:41, 58.27it/s]


254713it [53:41, 70.46it/s]


254726it [53:41, 81.24it/s]


254740it [53:41, 92.76it/s]


254752it [53:41, 67.59it/s]


254766it [53:42, 31.59it/s]


254778it [53:42, 40.31it/s]


254791it [53:42, 50.68it/s]


254801it [53:43, 50.13it/s]


254815it [53:43, 62.04it/s]


254825it [53:43, 65.51it/s]


254838it [53:43, 76.74it/s]


254852it [53:43, 88.30it/s]


254866it [53:43, 98.02it/s]


254879it [53:43, 104.97it/s]


254893it [53:43, 112.49it/s]


254907it [53:44, 117.50it/s]


254920it [53:44, 119.12it/s]


254935it [53:44, 125.58it/s]


254949it [53:44, 128.07it/s]


254963it [53:44, 115.68it/s]


254976it [53:44, 119.42it/s]


254991it [53:44, 125.40it/s]


255004it [53:44, 125.03it/s]


255019it [53:44, 129.83it/s]


255033it [53:45, 111.65it/s]


255047it [53:45, 118.36it/s]


255060it [53:45, 120.34it/s]


255073it [53:45, 122.68it/

Accuracy of the network on the test images: 1.899005 %


In [21]:
for key,value in snr_accuracy.items():
    print('{:s} dB SNR: accuracy: {:f}'.format(key, value[0]/value[1]))

6 dB SNR: accuracy: 0.784016
8 dB SNR: accuracy: 0.848516
10 dB SNR: accuracy: 0.891056
12 dB SNR: accuracy: 0.912698
14 dB SNR: accuracy: 0.922737
16 dB SNR: accuracy: 0.930349
18 dB SNR: accuracy: 0.933558
20 dB SNR: accuracy: 0.938976
22 dB SNR: accuracy: 0.937712
24 dB SNR: accuracy: 0.933947
26 dB SNR: accuracy: 0.939463
28 dB SNR: accuracy: 0.937780
30 dB SNR: accuracy: 0.938579
